# Interactive Machine Learning Pipelines with TFX

## Import Libraries

In [1]:
import os

# Do not show log that safe to ignore (you can see the message from log as well)
# See: https://github.com/tensorflow/tensorflow/issues/59779
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
import tensorflow_transform as tft
from modules.zoo import ModelDFZoo, InputFnZoo
import tensorflow_model_analysis as tfma
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Tuner, Trainer, Evaluator, Pusher
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

/home/hobiron/.pyenv/versions/3.8.0/envs/MLEnv/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Create Interactive Context Object

In [2]:
PIPELINE_NAME = "urge_pipeline"
# Directory to save input and output component (artifact)
PIPELINE_ROOT = os.path.join("pipelines", PIPELINE_NAME)
# Settings path of data
DATA_ROOT = os.path.join("data", "synthetic")

In [3]:
# The interactive object handles component execution and displays the component's artifacts
interactive_context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT
)

## Data Ingestion

In this step, we can read data files from local and remote folders (e.g., Google Cloud BigQuery). Before passing the ingested dataset to the next component, we divide the available data into separate datasets (e.g., training and validation datasets) and then convert the datasets into TFRecord format that containing the data represented as `tf.Example` data structures.

### Data Preparation (with Local Data Files)

The process of ingesting, splitting, and converting the dataset is performed by the 'ExampleGen' component. In this case, we have already generated the subsets of the datasets externally (check on data/synthetic_data_gen.ipynb file) and we would like to preverse these splits when we ingest the datasets.

#### Preserving Existing Splits

In [4]:
# Input train split is `DATA_ROOT/synth_train.csv`
# Input validation split is `DATA_ROOT/synth_val.csv`
# Output splits are generated one-to-one mapping from input splits
input = example_gen_pb2.Input(splits=[
    example_gen_pb2.Input.Split(name="train", pattern="synth_train.csv"),
    example_gen_pb2.Input.Split(name="val", pattern="synth_val.csv")
])

example_gen = CsvExampleGen(input_base=DATA_ROOT, input_config=input)

In [5]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [6]:
interactive_context.show(example_gen)

CsvExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x7f81accd04f0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7f81accd0640>, driver_class: <class 'tfx.components.example_gen.driver.FileBasedDriver'>, component_id: CsvExampleGen, inputs: {}, outputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None})

In [7]:
interactive_context.show(example_gen.outputs["examples"])

## Data Validation

### Check Statistics of The Data

We will use component called StatisticsGen to check statistical parameters of our data. This component accepts the output of the previous ExampleGen components as input.

In [8]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)

interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

We can visualize the output in interactive context by using:

In [9]:
interactive_context.show(statistics_gen)

StatisticsGen(spec: <tfx.types.standard_component_specs.StatisticsGenSpec object at 0x7f82287c93a0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7f818ca16610>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: StatisticsGen, inputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None}, outputs: {'statistics': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None})

In [10]:
interactive_context.show(statistics_gen.outputs["statistics"])

### Check Schema of The Data

Check that the data schema hasn't changed.

In [11]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [12]:
interactive_context.show(schema_gen)

SchemaGen(spec: <tfx.types.standard_component_specs.SchemaGenSpec object at 0x7f817d3e2550>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x7f817d3e21f0>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: SchemaGen, inputs: {'statistics': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None}, outputs: {'schema': OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None})

In [13]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'act_exe_num',INT,required,,-
'act_exe_nununique',INT,required,,-
'act_med_num',INT,required,,-
'act_med_nununique',INT,required,,-
'act_read_num',INT,required,,-
'act_read_nununique',INT,required,,-
'streak_current',INT,required,,-
'streak_status',INT,required,,-
'task_exe_completed',INT,required,,-


In this visualization above, `Presence` means whether the feature must be present in 100% of data examples (required) or not (optional). `Valency` means the number of values required per training example.

### Check Anomaly in The Data

In [14]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"]
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [15]:
interactive_context.show(example_validator)

ExampleValidator(spec: <tfx.types.standard_component_specs.ExampleValidatorSpec object at 0x7f817d3f5c40>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x7f817d3f5e20>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: ExampleValidator, inputs: {'statistics': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'schema': OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None}, outputs: {'anomalies': OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None})

In [16]:
interactive_context.show(example_validator.outputs["anomalies"])

Note:
The `ExampleValidator` can automatically detect the anomalies against the schema by using t[he skew and drift comparators](https://github.com/tensorflow/tfx/blob/cf04e1c99e7faf644bc91e498cf5cc58ea044eae/docs/tutorials/data_validation/tfdv_basic.ipynb). This may not cover all the potential anomalies in our data. If we want to detect some other specific anomalies, we need to write our own [custom component](https://www.tensorflow.org/tfx/guide/custom_component).

## Data Preprocessing

In [17]:
# We set environ variable to be consumed by transform_module.py
os.environ["PATH_SCHEMA"] = schema_gen.outputs["schema"]._artifacts[0].uri

transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath("modules/transform_module.py")
)

interactive_context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying utils.py -> build/lib
copying tuner_module.py -> build/lib
copying zoo.py -> build/lib
copying trainer_module.py -> build/lib
copying transform_module.py -> build/lib
installing to /tmp/tmp4prknqm_
running install
running install_lib
copying build/lib/utils.py -> /tmp/tmp4prknqm_
copying build/lib/tuner_module.py -> /tmp/tmp4prknqm_
copying build/lib/zoo.py -> /tmp/tmp4prknqm_
copying build/lib/trainer_module.py -> /tmp/tmp4prknqm_
copying build/lib/transform_module.py -> /tmp/tmp4prknqm_
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-inf


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


Processing ./pipelines/urge_pipeline/_wheels/tfx_user_code_Transform-0.0+2d0120744ce037ba9a7a3ff888b0776905469245b0687e9e7f09a3cf10056956-py3-none-any.whl



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


Processing ./pipelines/urge_pipeline/_wheels/tfx_user_code_Transform-0.0+2d0120744ce037ba9a7a3ff888b0776905469245b0687e9e7f09a3cf10056956-py3-none-any.whl


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [18]:
interactive_context.show(transform)

Transform(spec: <tfx.types.standard_component_specs.TransformSpec object at 0x7f817d3fba60>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7f817d3fbfa0>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Transform, inputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'schema': OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None}, outputs: {'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'post_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'post_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'post_transform_anomalies': OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None})

In [19]:
interactive_context.show(transform.outputs["transformed_examples"])
interactive_context.show(transform.outputs["transform_graph"])

The transformed data (transformed_examples) and the transform graph can be consumed during our next step, the `Tuner` and `Trainer component`.

## Model Development

Several studies have shown that the performance of tree-based models is still outperform deep learning on tabular data.  
See:
- [Why do tree-based models still outperform deep learning on tabular data?](https://arxiv.org/pdf/2207.08815.pdf)
- [Deep Neural Networks and Tabular Data: A Survey](https://arxiv.org/pdf/2110.01889)

By considering the studies results and with the required model compatibility in TFX. So the model that will be used in this case comes from the TensorFlow Decision Forest (TF-DF). TF-DF supports classification, regression and ranking. TF-DF makes it easy to train and use decision forests in the tensorflow ecosystem like TFX. Learn more: https://github.com/tensorflow/decision-forests

There are three type models that could be used on this case, either Random Forest, Gradient Boosted Trees or Cart Model. Before we go further (training or tuning) we will create fast test to determine best `input_fn` version and model with default parameters by accuracy and f1-score metrics.

**Important Notes:** Using these models have some caveats (helpfull for tuning & training process) ([source](https://github.com/tensorflow/decision-forests/blob/035529c1bc999d51caf25880d96e5813eef87736/tensorflow_decision_forests/keras/core.py#L326)):
- Decision Forest models are not Neural Networks. TF-Decision Forest is powered by Yggdrasil Decision Forest (YDF). Feature preprocessing that
  are beneficial to neural network (normalization, one-hot encoding) can be
  detrimental to decision forests. In most cases, it is best to feed the raw
  features (e.g. both numerical and categorical) without preprocessing to
  the model.
- During training, the entire dataset is loaded in memory (in an efficient
  representation). In case of large datasets (>100M examples), it is
  recommended to randomly downsample the examples.
- The model trains for exactly one epoch. The core of the training
  computation is done at the end of the first epoch. The console will show
  training logs (including validations losses and feature statistics).
- The model cannot make predictions before the training is done. Applying
  the model before training will raise an error. During training Keras
  evaluation will be invalid (the model always returns zero).
- Yggdrasil is itself a C++ model wrapper. Learners and models need to be
  added as dependency to the calling code. To make things practical, the
  Random Forest (without Borg distribution) and Gradient Boosted Decision
  Forest learners and models are linked by default. Other model/learners
  (including yours :)), needs to be added as a dependency manually.

In [20]:
os.environ["BATCH_SIZE"] = "128"
os.environ["LABEL_KEY"] = "streak_status"

In [21]:
obs_split = "train"
obs_artifacts = transform.outputs["transformed_examples"]._artifacts
obs_file_pattern = os.path.join(
    obs_artifacts[0].uri,
    f"Split-{obs_split}",
    "*"
)
obs_tf_transform_output = tft.TFTransformOutput(
    transform.outputs["transform_graph"]._artifacts[0].uri
)
obs_inputs = InputFnZoo(
    tf_transform_output=obs_tf_transform_output,
    batch_size=int(os.environ["BATCH_SIZE"]),
    transform_examples_artifacts=obs_artifacts
)
obs_inputs.compare(
    file_pattern=obs_file_pattern,
    label_key=os.environ["LABEL_KEY"],
    num_run=5
)

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


,Time Exec (ms) Run #1,Time Exec (ms) Run #2,Time Exec (ms) Run #3,Time Exec (ms) Run #4,Time Exec (ms) Run #5,Mean
Input Fn Version,,,,,,
V1,130.105892,57.034902,81.910326,66.572853,57.841834,78.693161
V2,883.956295,137.702274,119.158952,110.322391,106.494199,271.526822
V3,178.811946,102.414015,85.762472,76.247857,95.559633,107.759185


The table above help us to determine which one `input_fn` version that has minimum execution time, that will be usefull to handle big data later. And then th next one will use the best `input_fn` to compare the models.

In [22]:
# Learn more: modules/zoo.py
def get_dataset(split_type):
    return obs_inputs.get_dataset_with_best_input_fn(
        file_pattern=os.path.join(
            obs_artifacts[0].uri,
            f"Split-{split_type}",
            "*"
        ),
        label_key=os.environ["LABEL_KEY"]
    )

models = ModelDFZoo()
models.compare(
    train_ds=get_dataset("train"),
    valid_ds=get_dataset("val")
)

[INFO 23-08-25 13:36:46.4626 EDT kernel.cc:1242] Loading model from path /tmp/tmp2oh37adz/model/ with prefix 5945e3ac42094c5f
[INFO 23-08-25 13:36:46.6232 EDT decision_forest.cc:660] Model loaded with 300 root(s), 107356 node(s), and 10 input feature(s).
[INFO 23-08-25 13:36:46.6232 EDT abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 23-08-25 13:36:46.6233 EDT kernel.cc:1074] Use fast generic engine


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


[WARNING 23-08-25 13:37:06.1161 EDT gradient_boosted_trees.cc:1797] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-25 13:37:06.1162 EDT gradient_boosted_trees.cc:1808] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-25 13:37:06.1162 EDT gradient_boosted_trees.cc:1822] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 23-08-25 13:37:07.4468 EDT kernel.cc:1242] Loading model from path /tmp/tmpjl_92o42/model/ with prefix b3620149db0d451a
[INFO 23-08-25 13:37:07.4483 EDT abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-08-25 13:37:07.4484 EDT kernel.cc:1074] Use fast generic engine
[INFO 23-08-25 13:37:23.8546 EDT kernel.cc:1242] Loading model from path /tmp/tmpe1gzutvw/model/ with prefix cca24530651f4662
[INFO 23-08-25 13:37:23.8550 EDT decision_forest.cc:660] Model loaded with 1 root(s), 61 node(s), and 10 input feature(s).
[INFO 23-08-25 13:37:23.8550 EDT 

,Accuracy,Val Accuracy,AUC,Val AUC,Val F1 Score Micro,Val F1 Score Macro,Best Threshold
Model Name,,,,,,,
random_forest_model,0.918452,0.720833,0.977332,0.786530,0.729167,0.728105,0.55
gradient_boosted_trees_model,0.768452,0.741667,0.870572,0.792852,0.743056,0.743043,0.48
cart_model,0.772024,0.722222,0.816233,0.758957,0.740278,0.740217,0.61


For handle if the data imbalanced, the f1-score-macro metrics are used with a tuning threshold (note: for synthetic data already balanced but stil good to use Val F1 Score Macro). Threshold used to determine class prediction (binary) if probability of prediction in range [0, 1]. Note for accuracy and val accuracy metrics we still use threshold (0.5). For more detail read on: modules/zoo.py

Based on the result of the table above, model with the highest `Val F1 Score Macro` is `Gradient Boosted Trees Model (GBT)` 0.743043 with Threshold 0.48. And for R`andom Forest Model` is 0.728105 even though have the highest accuracy 0.918452, there is still have higher overfitting 0.918452 - 0.720833 = 0.197619 rather than `GBT Model` 0.768452 - 0.741667 = 0.026785. So for the next process we will use `GBT Model`.

In [23]:
models.mean_metrics

Accuracy              0.819643
Val Accuracy          0.728241
AUC                   0.888046
Val AUC               0.779446
Val F1 Score Micro    0.737500
Val F1 Score Macro    0.737122
dtype: float64

These mean of metrics will be used to evaluate our model later as min threshold metrics.

## Model Tuning

There are several keras tuners that can be used to tuning our model.  
see: https://keras.io/api/keras_tuner/

- RandomSearch Tuner  
This tuner choose randomly from all possible hyperparameter combinations
- GridSearch Tuner  
This tuner iterates over all possible hyperparameter combinations
- BayesianOptimization Tuner  
This tuner search best hyperparameter combinations with Gaussian process (need Sklearn library)
- Hyperband Tuner [Not suitable, epochs-based meanwhile TF-DF epochs=1]
- Sklearn Tuner [Not suitable, for sklearn models]

In this case, we will use GridSearch Tuner since the hyperparameters config only consist three parameter with the max number 10 * 3 * 3 = 90 combinations. See: `get_hyperparameters()` on `modules/tuner_module.py`.

In [24]:
# Options of Model
# RFM = Random Forest Model
# GBTM = Gradient Boosted Trees Model
# CM = Cart Model
os.environ["MODEL_TYPE"] = "GBTM"
os.environ["BEST_THRESHOLD"] = "0.48"
os.environ["INPUT_FN_VERSION"] = "V1"
# Options of Tuner
# GST = GridSearch Tuner
# RST = RandomSearch Tuner
# BOT = BayesianOptimization Tuner
os.environ["TUNER_TYPE"] = "GST"
os.environ["MAX_TRIALS"] = "30" # max 90 combinations

In [25]:
tuner = Tuner(
    examples=transform.outputs["transformed_examples"],
    schema=schema_gen.outputs["schema"],
    transform_graph=transform.outputs["transform_graph"],
    train_args=trainer_pb2.TrainArgs(splits=["train"]),
    eval_args=trainer_pb2.EvalArgs(splits=["val"]),
    module_file=os.path.abspath("modules/tuner_module.py")
)

interactive_context.run(tuner)

Trial 30 Complete [00h 00m 03s]
val_f1_score: 0.7483971118927002

Best val_f1_score So Far: 0.7580645084381104
Total elapsed time: 00h 01m 29s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines/urge_pipeline/.temp/6/untitled_project
Showing 10 best trials
Objective(name="val_f1_score", direction="max")

Trial 03 summary
Hyperparameters:
threshold: 0.48
num_trees: 250
max_depth: 4
min_examples: 6
Score: 0.7580645084381104

Trial 12 summary
Hyperparameters:
threshold: 0.48
num_trees: 255
max_depth: 4
min_examples: 6
Score: 0.7580645084381104

Trial 21 summary
Hyperparameters:
threshold: 0.48
num_trees: 260
max_depth: 4
min_examples: 6
Score: 0.7580645084381104

Trial 01 summary
Hyperparameters:
threshold: 0.48
num_trees: 250
max_depth: 5
min_examples: 7
Score: 0.756737470626831

Trial 10 summary
Hyperparameters:
threshold: 0.48
num_trees: 255
max_depth: 5
min_examples: 7
Score: 0.756737470626831

Trial 19 summary
Hyperparameters:
threshold: 0.48
num_trees: 260
max_depth: 5
min_examples: 7
Score: 0.756737470626831

Trial 28 summary
Hyperparameters:
threshold: 0.48
num_trees: 265
max_depth: 5
min_examples: 7
Score: 0.756737470626831

Trial 05 

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [26]:
interactive_context.show(tuner)

Tuner(spec: <tfx.types.standard_component_specs.TunerSpec object at 0x7f820d08a220>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x7f820d08a790>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Tuner, inputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'schema': OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None}, outputs: {'best_hyperparameters': OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'tuner_results': OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None})

## Model Training

The `TFX Trainer` component will produce a model that will be put into production, where it will transform new data and use the model to make predictions. Because the `Transform` steps are uncluded in this model, the data preprocessing steps will always match what the model is expecting. This removes a huge potential source of errors when our model is deployed.

In [27]:
trainer = Trainer(
    examples=transform.outputs["transformed_examples"],
    schema=transform.outputs["post_transform_schema"],
    transform_graph=transform.outputs["transform_graph"],
    hyperparameters=tuner.outputs["best_hyperparameters"],
    train_args=trainer_pb2.TrainArgs(splits=["train"]),
    eval_args=trainer_pb2.EvalArgs(splits=["val"]),
    module_file=os.path.abspath("modules/trainer_module.py")
)

interactive_context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying utils.py -> build/lib
copying tuner_module.py -> build/lib
copying zoo.py -> build/lib
copying trainer_module.py -> build/lib
copying transform_module.py -> build/lib
installing to /tmp/tmp03hqet84
running install
running install_lib
copying build/lib/utils.py -> /tmp/tmp03hqet84
copying build/lib/tuner_module.py -> /tmp/tmp03hqet84
copying build/lib/zoo.py -> /tmp/tmp03hqet84
copying build/lib/trainer_module.py -> /tmp/tmp03hqet84
copying build/lib/transform_module.py -> /tmp/tmp03hqet84
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.tx

Processing ./pipelines/urge_pipeline/_wheels/tfx_user_code_Trainer-0.0+2d0120744ce037ba9a7a3ff888b0776905469245b0687e9e7f09a3cf10056956-py3-none-any.whl



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


Reading training dataset...


[WARNING 23-08-25 13:39:16.6139 EDT gradient_boosted_trees.cc:1797] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-25 13:39:16.6139 EDT gradient_boosted_trees.cc:1808] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 23-08-25 13:39:16.6139 EDT gradient_boosted_trees.cc:1822] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines/urge_pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/urge_pipeline/Trainer/model/7/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [28]:
interactive_context.show(trainer)

Trainer(spec: <tfx.types.standard_component_specs.TrainerSpec object at 0x7f820cc4e160>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x7f820cc4e6a0>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Trainer, inputs: {'examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'hyperparameters': OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None}, outputs: {'model': OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, 'model_run': OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None})

## Model Analysis and Validation

Monitoring metrics through time before deployment, after deployment, and while in production is extremely important. Because the data that comes into the pipeline will change through time, often causing a decline in performance. In the TensorFlow ecosystem, there is TensorFlow Model Analysis (TFMA), which has all these capabilities.

### Resolve Baseline Model

Resolver is required component if we want to compate a new model against a previous model (in this case we use the latest_blessed_model_resolver) from metadata store. It checks for the last blessed (as baseline model), if there is no blessed, the Evaluator (later, check next step) will automatically blesses (plus if meet the metrics treshold) the model (as baseline model).

In [29]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id("latest_blessed_model_resolver")

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [30]:
interactive_context.show(model_resolver)

### Validation with Evaluator

Evaluator is component uses the TFMA to evaluate our model's predictions on a validation dataset. It takes as input data from the transformed_examples of Transform component, the trained model from the Trainer component (as new model), and EvalConfig for TFMA. Evaluator also carries out validation, it will checks whether the new model is an improvement on a baseline model. It obtains predictions from both model on evaluation dataset and compares performance metrics. If the new model is an improvement on the previous model (baseline), the new model will receives a `"blessing"` artifact.

In [31]:
# Currently it is only possible to calculate the metric (blessing) 
# on the whole evalution set, not on slices, for example: threshold min metric = 0.8
# value metric on feature "A" (target slices) is 0.89 meanwhile 
# value metric on whole eval set is 0.7 (then the model will be not blessed)
eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name="eval_for_tfma",
            label_key=os.environ["LABEL_KEY"],
            preprocessing_function_names=["transform_features"]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec()
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # See: https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/metrics
            # See: https://github.com/tensorflow/model-analysis/blob/v0.44.0/tensorflow_model_analysis/proto/config.proto
            metrics=[
                tfma.MetricConfig(
                    # Bug: when use average macro but OK when use average micro
                    # See: https://discuss.tensorflow.org/t/cannot-update-variable-with-shape-2-using-a-tensor-with-shape-512-1-shapes-must-be-equal-op-assignaddvariableop/5071
                    module="utils",
                    class_name="F1ScoreBinaryBridge",
                    config=f'"average": "micro", "name": "f1_score_micro", "threshold": {os.environ["BEST_THRESHOLD"]}',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            # For synthetic data still good even use micro since the data already balanced
                            lower_bound={"value": models.mean_metrics["Val F1 Score Micro"]}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": 0.01}
                        )
                    )
                ),
                tfma.MetricConfig(
                    class_name="AUC",
                    config='"curve": "ROC"',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": models.mean_metrics["Val AUC"]}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": 0.01}
                        )
                    )
                ),
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    # We will use default threshold (0.5) since the lower bound use it as well
                    # See ModelZooDF on modules/zoo.py module
                    # config=f'"threshold": {os.environ["BEST_THRESHOLD"]}',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": models.mean_metrics["Val Accuracy"]}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": 0.01}
                        )
                    )
                ),
                # Necessary metrics for binary classification case
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name='FalsePositives'),
                tfma.MetricConfig(class_name='TruePositives'),
                tfma.MetricConfig(class_name='FalseNegatives'),
                tfma.MetricConfig(class_name='TrueNegatives')
            ],
        )
    ]
)

In [32]:
evaluator = Evaluator(
    examples=example_gen.outputs["examples"],
    example_splits=["train", "val"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config
)

interactive_context.run(evaluator)

[INFO 23-08-25 13:39:31.7858 EDT kernel.cc:1242] Loading model from path pipelines/urge_pipeline/Trainer/model/7/Format-Serving/assets/ with prefix 1a5e15b39f6c43e4
[INFO 23-08-25 13:39:31.7869 EDT abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-08-25 13:39:31.7870 EDT kernel.cc:1074] Use fast generic engine


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [33]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'f1_score_mi…

In [34]:
print(tfma.load_validation_result(eval_result))
blessing_uri = evaluator.outputs["blessing"]._artifacts[0].uri
!ls -l {blessing_uri}

validation_ok: true
validation_details {
  slicing_details {
    slicing_spec {
    }
    num_matching_slices: 1
  }
}

total 0
-rw-r--r-- 1 hobiron hobiron 0 Aug 25 13:39 BLESSED


The `Evaluator` has blessed the model (the model is an improvement on the previous version, and it is above any thresholds (AUC, Binary Accuracy, and F1 Score Micro) that we have set).

## Push Model Blessed

If our model has been blessed, the Pusher component will push the model to the serving file path (still in local).

In [35]:
pusher = Pusher(
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=os.path.join("serving_saved_model", "urge_model")
        )
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Once the new model has been pushed to the serving directory, it can then be picked up by TensorFlow Serving for deployment.

## References

- [TensorFlow Extended (TFX) Tutorials](https://www.tensorflow.org/tfx/tutorials)
- [TFX Guide & Docs](https://www.tensorflow.org/tfx/guide)
- [TFX Examples (GitHub)](https://github.com/tensorflow/tfx/tree/master/tfx/examples)
- [Oreily Ebook MLOps TFX](https://www.oreilly.com/library/view/building-machine-learning/9781492053187/)
- [Dicoding MLOps Course](https://www.dicoding.com/academies/443)